### Libraries

In [2]:
from typing import Union
import numpy as np
from qclib.gates.mcu import MCU
from qclib.gates.ldmcu import Ldmcu
from qiskit import (QuantumCircuit,
                    QuantumRegister,
                    transpile)
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.quantum_info import state_fidelity, Statevector, DensityMatrix
from qiskit_aer import AerSimulator
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
import matplotlib.pyplot as plt

In [3]:
class Utilities:

    def __init__(self, operator, error):
        self.operator = operator
        self.error = error

    @staticmethod
    def pauli_matrices(string: str) -> Union[np.ndarray, None]:
        string_case = string.lower()
        match string_case:
            case "x":
                return np.array([
                    [0, 1],
                    [1, 0]
                ])
            case "y":
                return np.array([
                    [0, -1j],
                    [1j, 0]
                ])
            case "z":
                return np.array([
                    [1, 0],
                    [0, -1]
                ])
            case _:
                return None

    @staticmethod
    def transpose_conjugate(operator):
        return np.conjugate(np.transpose(operator))

    @staticmethod
    def pyramid_size(operator, error):
        mcu_dummy = MCU(operator, num_controls=100, error=error)
        # will be changed by mcu_dummy._get_n_base(x_dagger, error)
        return mcu_dummy._get_num_base_ctrl_qubits(operator, error)


In [4]:
def built_circuit(extra_qubits=0, pauli_string='x', error=0.1, approximated=True):
    
    pauli_matrix = Utilities.pauli_matrices(pauli_string)
    pauli_matrix_dagger = Utilities.transpose_conjugate(pauli_matrix)
    n_base = Utilities.pyramid_size(pauli_matrix_dagger, error)

    controls = QuantumRegister(n_base + extra_qubits, 'controls')
    target = QuantumRegister(1, 'target')
    circ = QuantumCircuit(controls, target)
    circ.x(list(range(len(controls))))
    
    if approximated:
        MCU.mcu(circ, pauli_matrix_dagger, controls, target, error)
    else:
        Ldmcu.ldmcu(circ, pauli_matrix_dagger, controls, target)
        
    # circ.measure_all()
    
    return circ

In [22]:
def create_noise_model():
    #noise_model = NoiseModel.from_backend(backend)
    # Create an empty noise model
    noise_model = NoiseModel()

    # Add depolarizing error to all single qubit u1, u2, u3 gates
    error_1qubit = depolarizing_error(0.0001, 1)
    error_2qubit = depolarizing_error(0.001, 2)
    noise_model.add_all_qubit_quantum_error(error_1qubit, ['u1', 'u2', 'u3'])
    noise_model.add_all_qubit_quantum_error(error_2qubit, ['cx'])
    # Measurement miss-assignement probabilities (No Effect)
    #p0given1 = 0.5
    #p1given0 = 0.5
    #readout = ReadoutError([[1 - p1given0, p1given0], [p0given1, 1 - p0given1]])
    #noise_model.add_all_qubit_readout_error(readout)
    return noise_model



In [23]:
def density_matrix(circuit):
    
    noise_model = create_noise_model()
    simulator = AerSimulator(noise_model=noise_model, method='density_matrix')
    transpiled_circuit = transpile(circuit, simulator, basis_gates= noise_model.basis_gates)
    transpiled_circuit.save_density_matrix()
    
    job = simulator.run(transpiled_circuit)
    result = job.result()
    d_m = result.data(0)
    
    return d_m

In [24]:
def density_matrix_noiseless(circuit):
    
    size = len(circuit)
    simulator = AerSimulator( method='density_matrix')
    transpiled_circuit = transpile(circuit, simulator)
    transpiled_circuit.save_density_matrix()
    
    job = simulator.run(transpiled_circuit)
    result = job.result()
    d_m = result.data(0)

    return d_m

### Novos experimentos

In [28]:
#Se for útil, deixamos isso tudo como uma função de M e n_extra
#M = 3
n_extra = [0,1,2,3]
fidelity_exact = np.zeros((len(n_extra)))
fidelity_approx = np.zeros((len(n_extra)))

error = 0.1
matrix = 'x'

for i in range(len(n_extra)):
    ne = n_extra[i]
    approx_circuit = built_circuit(ne, matrix, error)
    real_circuit = built_circuit(ne, matrix, error, False)

    t_array = np.zeros(2**len(real_circuit))
    t_array[-1] = 1
    t_dm = DensityMatrix(t_array)
    print("Ne = ", ne)

    #for k in range(M):
    exact_decomposition_dm = density_matrix(real_circuit)
    approximated_decomposition_dm = density_matrix(approx_circuit)
    fidelity_exact[ne] = state_fidelity(t_dm, exact_decomposition_dm['density_matrix'])
    fidelity_approx[ne] = state_fidelity(t_dm, approximated_decomposition_dm['density_matrix'])


    print("Exact", fidelity_exact[i])
    print("Approx", fidelity_approx[i])

Ne =  0
Exact 0.9060109763562219
Approx 0.9053280547091812
Ne =  1
Exact 0.8717731227154056
Approx 0.8909534510782845
Ne =  2
Exact 0.8335155914634449
Approx 0.8625675999139574
Ne =  3
Exact 0.7918943140297156
Approx 0.8350610099377931


### Experimentos antigos

In [18]:
#Se for útil, deixamos isso tudo como uma função de M e n_extra
M = 30
n_extra = [0,1,2,3]
fidelity_exact = np.zeros((len(n_extra), M))
fidelity_noisy = np.zeros((len(n_extra), M))

error = 0.1
matrix = 'x'

for i in range(len(n_extra)):
    ne = n_extra[i]
    approx_circuit = built_circuit(ne, matrix, error)
    real_circuit = built_circuit(ne, matrix, error, False)

    t_array = np.zeros(2**len(real_circuit))
    t_array[-1] = 1
    t_dm = DensityMatrix(t_array)
    print("Ne = ", ne)

    for k in range(M):
        exact_decomposition_dm = density_matrix(real_circuit)
        approximated_decomposition_dm = density_matrix(approx_circuit)
        fidelity_exact[ne][k] = state_fidelity(t_dm, exact_decomposition_dm['density_matrix'])
        fidelity_noisy[ne][k] = state_fidelity(t_dm, approximated_decomposition_dm['density_matrix'])


    print(fidelity_exact[i].mean(), fidelity_exact[i].std())
    print(fidelity_noisy[i].mean(), fidelity_noisy[i].std())


Ne =  0
0.5907192704008323 0.02913509344287548
0.5891957930721783 0.03373324101788872
Ne =  1
0.4915427464348829 0.024711643650861245
0.5293876118614685 0.03917868161620722
Ne =  2
0.40097801969852404 0.02587425016326948
0.45371054636893793 0.03249792249700125
Ne =  3
0.3046776029021234 0.025662763676235677
0.38025235756133846 0.033575292671178196


### Noiseless

In [77]:
t_array = np.zeros(2**n1)
t_array[-1] = 1
t_dm = DensityMatrix(t_array)
exact_decomposition_dm, n1 = density_matrix_noiseless(real_circuit)
approximated_decomposition_dm, n2 = density_matrix_noiseless(approx_circuit)
print(state_fidelity(t_dm, exact_decomposition_dm['density_matrix']))
print(state_fidelity(t_dm, approximated_decomposition_dm['density_matrix']))

1.000000000000031
0.9975923633361293


### State Vector (OLD)

In [ ]:
def state_vector(circuit):
    
    size = len(circuit)
    backend = GenericBackendV2(num_qubits=size, seed=42)
    noise_model = NoiseModel.from_backend(backend)
    
    simulator = AerSimulator(noise_model=noise_model)
    transpiled_circuit = transpile(circuit, simulator, basis_gates= noise_model.basis_gates)
    transpiled_circuit.save_statevector()
    
    job = simulator.run(transpiled_circuit)
    result = job.result()
    sv = result.get_statevector()
    # result = job.result().get_counts(0)
    # d_m = DensityMatrix(transpiled_circuit)
    return sv, size

In [ ]:
def state_vector_noiseless(circuit):
    
    size = len(circuit)
    
    simulator = AerSimulator()
    transpiled_circuit = transpile(circuit, simulator)
    transpiled_circuit.save_statevector()
    
    job = simulator.run(transpiled_circuit)
    result = job.result()
    sv = result.get_statevector()
    return sv, size

In [72]:
exact_decomposition_s_vector, n1 = state_vector(real_circuit)
t_array = np.zeros(2**n1)
t_array[-1] = 1
t_state_vector = Statevector(t_array)
approximated_decomposition_s_vector, n2 = state_vector(approx_circuit)
print(state_fidelity(t_state_vector, exact_decomposition_s_vector))
print(state_fidelity(t_state_vector, approximated_decomposition_s_vector))

0.9998062260475085
2.166108588537142e-28


In [51]:
exact_decomposition_s_vector, n1 = state_vector_noiseless(real_circuit)
t_array = np.zeros(2**n1)
t_array[-1] = 1
t_state_vector = Statevector(t_array)
approximated_decomposition_s_vector, n2 = state_vector_noiseless(approx_circuit)
print(state_fidelity(t_state_vector, exact_decomposition_s_vector))
print(state_fidelity(t_state_vector, approximated_decomposition_s_vector))

1.000000000000004
0.9975923633361053
